In [1]:
import scipy.sparse as spr
import scipy.sparse.linalg as spla
import sys
path2oti = '../../../../build/'
sys.path.append(path2oti) # Add path to OTI library.

import pyoti.real   as r
import pyoti.sparse as oti 
import pyoti.core   as coti
import pyoti.fem    as fem 

# import pyoti.static.onumm1n1    as dual
# import pyoti.static.mdnum2      as md2
# import pyoti.static.mdnum3      as md3
# import pyoti.static.mdnum5      as md5
# import pyoti.static.mdnum6      as md6
# import pyoti.static.mdnum10     as md10
# import pyoti.static.onumm1n10   as om1n10
# import pyoti.static.onumm1n2    as om1n2
# import pyoti.static.onumm1n5    as om1n5
# import pyoti.static.onumm5n5    as om5n5
# import pyoti.static.onumm2n2    as om2n2
# import pyoti.static.onumm3n3    as om3n3
# import pyoti.static.onumm2n10    as om2n10




%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

e  = oti.e
np = oti.np

import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')

global times
from timeit import default_timer as time

from matplotlib import rc

## for Palatino and other serif fonts use:
rc('font',**{'family':'serif','serif':['Palatino'], 'size':16})
rc('text', usetex=True)

In [2]:
#*****************************************************************************************************
def solve_2d_cavity(Th, utop, stats=True, solver = 'SuperLU'):
    
    global times
    from timeit import default_timer as time
    
    print("Setting up problem")
    start_time = time()
    
    c1 = -1e-10 # Constant for p*q term 
    
    ndim_analysis = 2
    els = Th.elements[2]

    fem.end_elements()
    
    nnodes_tri6 = Th.nnodes
    nnodes_tri3 = np.unique(Th.elements[2]['indices'][0][:,:3]).size
    
    # Create an index map for use in the matrix creation.
    unique = np.unique(Th.elements[2]['indices'][0][:,:3])
    ind_map = np.ones(Th.nnodes,dtype=int)*-1

    ind_map[unique] = np.arange(unique.size)
    
    nDOF   = 2*nnodes_tri6 + nnodes_tri3 # 
    
    
    K = alg.lil_matrix((nDOF,nDOF))
    f = alg.zeros((nDOF,1))
    
    
    print("\nStarting elemental computations.")
    
    # 
    for j in range(els['types'].size):

        elm_o2 = fem.element_list[ els['types'][j] ][2] # Take order 2
        elm_o1 = fem.element_list[ els['types'][j] ][1] # Take order 1

        if (not elm_o2.is_allocated()) or (not elm_o1.is_allocated()):

            elm_o2.end()
            elm_o1.end()
            
            elm_o2.allocate(intorder=2)
            elm_o2.allocate_spatial(ndim_analysis,compute_Jinv = True)
            
            elm_o1.allocate(intorder=2)
            elm_o1.allocate_spatial(ndim_analysis,compute_Jinv = True)

            # Temps
            
            UxT = alg.zeros( ( elm_o2.nbasis, 1 ), nip = elm_o2.nip )
            UyT = alg.zeros( ( elm_o2.nbasis, 1 ), nip = elm_o2.nip )
            UT  = alg.zeros( ( elm_o2.nbasis, 1 ), nip = elm_o2.nip )
            
            UxUx = alg.zeros( ( elm_o2.nbasis, elm_o2.nbasis ), nip = elm_o2.nip )
            UyUy = alg.zeros( ( elm_o2.nbasis, elm_o2.nbasis ), nip = elm_o2.nip )

            UU_tmp1 = alg.zeros( ( elm_o2.nbasis, elm_o2.nbasis ), nip = elm_o2.nip )
            
            PT  = alg.zeros( ( elm_o1.nbasis, 1 ), nip = elm_o2.nip )
            PxT = alg.zeros( ( elm_o1.nbasis, 1 ), nip = elm_o2.nip )
            PyT = alg.zeros( ( elm_o1.nbasis, 1 ), nip = elm_o2.nip )
            
            PP = alg.zeros( ( elm_o1.nbasis, elm_o1.nbasis ), nip = elm_o1.nip )
            PP_tmp1 = alg.zeros( ( elm_o1.nbasis, elm_o1.nbasis ), nip = elm_o1.nip )
                        
            UPx = alg.zeros( ( elm_o2.nbasis, elm_o1.nbasis ), nip = elm_o2.nip )
            UPy = alg.zeros( ( elm_o2.nbasis, elm_o1.nbasis ), nip = elm_o2.nip )
            
            
            PUx = alg.zeros( ( elm_o1.nbasis, elm_o2.nbasis ), nip = elm_o2.nip )
            PUy = alg.zeros( ( elm_o1.nbasis, elm_o2.nbasis ), nip = elm_o2.nip )
                        
            tmp11 = alg.zeros( ( elm_o2.nbasis, elm_o2.nbasis ) )
            tmp22 = alg.zeros( ( elm_o2.nbasis, elm_o2.nbasis ) )
            
            tmp13 = alg.zeros( ( elm_o2.nbasis, elm_o1.nbasis ) )
            tmp23 = alg.zeros( ( elm_o2.nbasis, elm_o1.nbasis ) )
            
            tmp31 = alg.zeros( ( elm_o1.nbasis, elm_o2.nbasis ) )
            tmp32 = alg.zeros( ( elm_o1.nbasis, elm_o2.nbasis ) )
            
            tmp33 = alg.zeros( ( elm_o1.nbasis, elm_o1.nbasis ) )

        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            elm_o2.set_coordinates(Th.x,Th.y,Th.z,elems)
            elm_o2.compute_jacobian()
            
            elm_o1.set_coordinates(Th.x,Th.y,Th.z,elems)
            elm_o1.compute_jacobian()

            Ux = elm_o2.Nx
            Uy = elm_o2.Ny
            U  = elm_o2.N
            
            Px = elm_o1.Nx
            Py = elm_o1.Ny
            P  = elm_o1.N
            
            alg.transpose(U,  out = UT )
            alg.transpose(Ux, out = UxT)
            alg.transpose(Uy, out = UyT)
            
            alg.transpose(P,  out = PT )
            alg.transpose(Px, out = PxT)
            alg.transpose(Py, out = PyT)
            
            alg.dot( UxT, Ux, out = UxUx )
            alg.dot( UyT, Uy, out = UyUy )
            
            alg.dot( PT, P, out = PP )
            
            alg.dot( PT, Ux, out = PUx )
            alg.dot( PT, Uy, out = PUy )
            
            alg.dot( UT, Px, out = UPx )
            alg.dot( UT, Py, out = UPy )
                
            # solve stokes([ux,uy,p],[vx,vy,q]) = 
            #     int2d(Th)( dx(ux)*dx(vx) + dy(ux)*dy(vx)  ) + 
            #     int2d(Th)( dx(uy)*dx(vy) + dy(uy)*dy(vy)  ) + 
            #     int2d(Th)( dx(p)*vx      + dy(p)*vy       ) + 
            #     int2d(Th)( q*dx(ux)      + q*dy(uy)       ) + 
            #     int2d(Th)( (-1e-10)*p*q                   ) 
            
            
            alg.sum( UxUx, UyUy, out= UU_tmp1 )
            alg.gauss_integrate( UU_tmp1, elm_o2.dV, out = tmp11 ) # vx,ux
            
            alg.gauss_integrate( UU_tmp1, elm_o2.dV, out = tmp22 ) # vy,uy
            
            alg.gauss_integrate( UPx, elm_o2.dV, out = tmp13 ) # dx(p)*vx
            alg.gauss_integrate( UPy, elm_o2.dV, out = tmp23 ) # dy(p)*vy
            
            alg.gauss_integrate( PUx, elm_o2.dV, out = tmp31 ) # q*dx(ux)
            alg.gauss_integrate( PUy, elm_o2.dV, out = tmp32 ) # q*dy(uy)
            
            alg.mul( c1, PP, out = PP_tmp1 )
            alg.gauss_integrate( PP_tmp1, elm_o2.dV, out = tmp33 ) # 
            
            # assemble globals
            
            for k in range(elm_o2.nbasis):
        
                ii=int(elems[k])
                
                i1 = ii
                i2 = ii + nnodes_tri6 
                
                for l in range(elm_o2.nbasis):

                    jj=int(elems[l])
                    
                    j1 = jj
                    j2 = jj + nnodes_tri6 
                    
                    K[i1,j1] = K[i1,j1] + tmp11[k,l]                    
                    K[i2,j2] = K[i2,j2] + tmp22[k,l]
                    
                # end for 

            # end for 

            for k in range(elm_o2.nbasis):
        
                ii=int(elems[k])
            
                i1 = ii
                i2 = ii + nnodes_tri6 
                i3 = int(ind_map[ii] + 2*nnodes_tri6)
                
                for l in range(elm_o2.nbasis):

                    jj=int(elems[l])
                    
                    j1 = jj
                    j2 = jj + nnodes_tri6 
                    j3 = int(ind_map[jj] + 2*nnodes_tri6 )
                    
                    if l < elm_o1.nbasis:
                        K[i1,j3] = K[i1,j3] + tmp13[k,l]
                        K[i2,j3] = K[i2,j3] + tmp23[k,l]
                    
                        if k < elm_o1.nbasis:
                            K[i3,j3] = K[i3,j3] + tmp33[k,l]
                        #
                    
                    #
                    if k < elm_o1.nbasis:
                        K[i3,j1] = K[i3,j1] + tmp31[k,l]
                        K[i3,j2] = K[i3,j2] + tmp32[k,l]
                    #
                    

                # end for 

            # end for 
 
        # end for

    # end for

    fem.end_elements()   

    end_assmbly_time = time()
    print("\nFinished assembly.\nSetting up boundary condition.")
    
    # Setting Dirichlet BCs using TGV.
    TGV = 1e30
    
    # Every 1D node has dirichlet bc = 0
    els = Th.group_names['right']['members'][0]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            i1 = int(ii_)
            i2 = int(ii_) + nnodes_tri6

            K[i1,i1] = TGV
            f[i1,0]  = 0.0
            
            K[i2,i2] = TGV
            f[i2,0]  = 0.0
            
        # end for 
                
    # end for
    
    
    # Every 1D node has dirichlet bc = 0
    els = Th.group_names['left']['members'][0]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            i1 = int(ii_)
            i2 = int(ii_) + nnodes_tri6

            K[i1,i1] = TGV
            f[i1,0]  = 0.0
            
            K[i2,i2] = TGV
            f[i2,0]  = 0.0
            
        # end for 
                
    # end for
    
    
    # Every 1D node has dirichlet bc = 0
    els = Th.group_names['bottom']['members'][0]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            i1 = int(ii_)
            i2 = int(ii_) + nnodes_tri6

            K[i1,i1] = TGV
            f[i1,0]  = 0.0
            
            K[i2,i2] = TGV
            f[i2,0]  = 0.0
            
        # end for 
                
    # end for
    
    # Every 1D node has dirichlet bc = 0
    els = Th.group_names['top']['members'][0]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            i1 = int(ii_)
            i2 = int(ii_) + nnodes_tri6

            K[i1,i1] = TGV
            f[i1,0]  = utop*TGV
            
            K[i2,i2] = TGV
            f[i2,0]  = 0.0
            
        # end for 
                
    # end for
    
#     # pressure =0 at bottom left corner
#     els = Th.group_names['bottomleft']['members'][0]
#     for j in range(els['types'].size):

#         elm_nodes = np.unique(els['indices'][j])

#         for ii_ in elm_nodes:
#             i1 = int(ii_)
#             i2 = int(ii_) + nnodes_tri6
#             i3 = int(ind_map[ii] + 2*nnodes_tri6)

#             K[i3,i3] = TGV
#             f[i3,0]  = 0
            
            
#         # end for 
                
#     # end for
    
    
    
    end_bc_time = time()
    
    print("\nEnded boundary condition setup.\nStarting system solution.")
    
    u = 0
    K = K.tocsr()#(preserve_in=False)
    
    print("\nConverted matrix to csr format. - Starting solution process.")
    
    u = alg.solve(K,f,solver=solver)
    
    print("\nSolved problem.")
    
    end_solve_time = time()
    
    if stats:
        times['assembly'].append(  end_assmbly_time - start_time       )
        times['bc'].append( end_bc_time      - end_assmbly_time  )
        times['solve'].append(  end_solve_time   - end_bc_time      )
        times['total'].append( end_solve_time   - start_time        )
        
#         print("Assembly time:  {0:.6f} s ".format( end_assmbly_time - start_time       ) )
#         print("Boundary time:  {0:.6f} s ".format( end_bc_time      - end_assmbly_time ) )
#         print("Solution time:  {0:.6f} s ".format( end_solve_time   - end_bc_time      ) )
        print("Total run time: {0:.6f} s ".format( end_solve_time   - start_time       ) )
        print()
    # end if 
    
    ux = u[:nnodes_tri6]
    uy = u[nnodes_tri6:2*nnodes_tri6]
    p  = u[2*nnodes_tri6:]

    return [ux,uy,p],K,f

    #-----------------------------------------------------------------------------------------------------

In [3]:
#*****************************************************************************************************
def convert_to_o2(Th, val):
    
    els = Th.elements[2]

    fem.end_elements()
    
    nnodes_tri6 = Th.nnodes
    nnodes_tri3 = np.unique(Th.elements[2]['indices'][0][:,:3]).size
    
    # Create an index map for use in the matrix creation.
    unique = np.unique(Th.elements[2]['indices'][0][:,:3])
    ind_map = np.ones(Th.nnodes,dtype=int)*-1

    ind_map[unique] = np.arange(unique.size)
    
    np.ones(nnodes_tri6)
    res = alg.zeros((nnodes_tri6,1))
    
    for j in range(els['types'].size):

        elm_o2 = fem.element_list[ els['types'][j] ][2] # Take order 2
        elm_o1 = fem.element_list[ els['types'][j] ][1] # Take order 1

        if (not elm_o2.is_allocated()) or (not elm_o1.is_allocated()):

            elm_o2.end()
            elm_o1.end()
            
            elm_o2.allocate(intorder=2)            
            elm_o1.allocate(intorder=2)

            
        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            # print("\n\n Element {0}.".format(i))
            # print(elems)
            
            for k in range(3):
                
                i1 = int(elems[k])
                ii1= int(ind_map[i1])
                i2 = int(elems[(k+1)%3])
                ii2= int(ind_map[i2])
                
                j1 = int(elems[k+3])
                
                # print( "i1: {0}, ii1: {1}, i2: {2}, ii2: {3}, j1: {4}".format(i1,ii1,i2,ii2,j1) )
                
                
                res[i1,0] = val [ii1,0]
                
                res[j1,0] = 0.5*( val [ii1,0] + val [ii2,0] )
                
            # end for 
        
        # end for 

    # end for

    fem.end_elements()   

    return res

    #-----------------------------------------------------------------------------------------------------
    
#*****************************************************************************************************
def convert_to_o1(Th, val):
    
    els = Th.elements[2]

    fem.end_elements()
    
    nnodes_tri6 = Th.nnodes
    nnodes_tri3 = np.unique(Th.elements[2]['indices'][0][:,:3]).size
    
    # Create an index map for use in the matrix creation.
    unique = np.unique(Th.elements[2]['indices'][0][:,:3])
    ind_map = np.ones(Th.nnodes,dtype=int)*-1

    ind_map[unique] = np.arange(unique.size)
    
    np.ones(nnodes_tri6)
    res = alg.zeros((nnodes_tri3,1))
    
    for j in range(els['types'].size):

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            elems = elm_nodes[i,:]
            
            # print("\n\n Element {0}.".format(i))
            # print(elems)
            
            for k in range(3):
                
                i1 = int(elems[k])
                ii1= int(ind_map[i1])
                
                res[ii1,0] = val [i1,0]
                
            # end for 
        
        # end for 

    # end for

    fem.end_elements()   

    return res

    #-----------------------------------------------------------------------------------------------------

In [19]:
# fem.set_global_algebra(r)
fem.set_global_algebra(oti)
# fem.set_global_algebra(dual)
alg = fem.get_global_algebra()

start_time = time()

hx = 1
hy = 1

Th = fem.square(hx,hy,he=0.01,element_order=2,quads=False,save=False, structured=True)
Th2= fem.square(hx,hy,he=0.01,element_order=2,quads=False,save=False, structured=True)

end_time = time()

print(end_time - start_time,"seg")
print(Th)
# move the mesh
Th.x += hx/2
Th.y += hy/2

0.2607831570203416 seg
< mesh (pyoti.sparse) object with 40401 nodes, 20404 elements of types: point1 (4), line3 (400), tri6 (20000) >


In [20]:
Th.group_names.keys()

dict_keys(['topleft', 'bottomleft', 'topright', 'bottomright', 'left', 'bottom', 'right', 'top', 'domain'])

In [21]:
order = 30


utop = alg.number(1)#+alg.e(2, order = order)

solver = 'umfpack'

# # Perturb nodal coordinates for ro
# pc = 1.0
# xmax = hx
# xmin = 0
# for i in range(Th.x.size):
    
#     x = Th.x[i,0]
    
#     if x.real > (xmax-pc*(xmax-xmin)):
        
#         h = (1-(xmax-x)/(pc*(xmax-xmin))).real

#         Th.x[i,0] = x.real + h*alg.e(1, order = order)
        
#     else:
        
#         Th.x[i,0] = x.real
    
#     # end if 
# # end for 

# ymax = hy
# ymin = 0
# for i in range(Th.y.size):
    
#     y = Th.y[i,0]
    
#     if y.real > (ymax-pc*(ymax-ymin)):
        
#         h = (1-(ymax-y)/(pc*(ymax-ymin))).real

#         Th.y[i,0] = y.real + h*alg.e(2, order = order)
        
#     else:
        
#         Th.y[i,0] = y.real
    
#     # end if 
    
# # end for 


In [22]:
times = {}
times['assembly'] = []
times['bc'] = []
times['solve'] = []
times['total'] = []



for i in range(1):
    [ux, uy, pf ],K,f = solve_2d_cavity(Th, utop, solver=solver)
#end for

print('\n\nTimes:'+str(type(ux)))
print("- Avg Assembly time:  {0:.6f} s ".format(np.average(times['assembly'] ) ) )
print("- Avg Boundary time:  {0:.6f} s ".format(np.average(times['bc'])      ) )
print("- Avg Solution time:  {0:.6f} s ".format(np.average(times['solve'])   ) )
print("- Avg Total run time: {0:.6f} s ".format(np.average(times['total'])   ) )

Setting up problem

Starting elemental computations.

Finished assembly.
Setting up boundary condition.

Ended boundary condition setup.
Starting system solution.

Converted matrix to csr format. - Starting solution process.

Solved problem.
Total run time: 7.015301 s 



Times:<class 'pyoti.sparse.matso'>
- Avg Assembly time:  5.914823 s 
- Avg Boundary time:  0.001574 s 
- Avg Solution time:  1.098904 s 
- Avg Total run time: 7.015301 s 


In [8]:
K

<(91003, 91003) sparse matrix of OTI numbers with 
         1757807 stored elements in Compressed Sparse Row format>

In [1]:
K[1000,1000].short_repr()

NameError: name 'K' is not defined

### Convert Pressure array to Order 2 interpolation.

In [9]:
pf_o2 = convert_to_o2(Th,pf)

In [10]:
pf_o2.real

array([[-5.52459190e+02],
       [-3.36969423e-01],
       [ 5.10670684e+02],
       ...,
       [ 3.36550830e-01],
       [ 3.36628222e-01],
       [ 3.37396371e-01]])

### Convert results to single matrix.


In [14]:
u = alg.zeros((ux.shape[0],3))
u[:,0:1] = ux
u[:,1:2] = uy

u[40000,0]

-0.0048 - 0.0181 * e([1]) + 0.0181 * e([2]) - 0.0021 * e([[1,2]]) + 0.0223 * e([1,2]) - 0.0202 * e([[2,2]]) + 0.0272 * e([[1,3]]) - 0.0774 * e([[1,2],2]) + 0.0552 * e([1,[2,2]]) - 0.0049 * e([[2,3]]) - 0.0292 * e([[1,4]]) + 0.0353 * e([[1,3],2]) + 0.0632 * e([[1,2],[2,2]]) - 0.0973 * e([1,[2,3]]) + 0.0280 * e([[2,4]]) - 0.0075 * e([[1,5]]) + 0.1541 * e([[1,4],2]) - 0.3787 * e([[1,3],[2,2]]) + 0.2944 * e([[1,2],[2,3]]) - 0.0499 * e([1,[2,4]]) - 0.0124 * e([[2,5]]) + 0.0590 * e([[1,6]]) - 0.3169 * e([[1,5],2]) + 0.4070 * e([[1,4],[2,2]]) + 0.0886 * e([[1,3],[2,3]]) - 0.4345 * e([[1,2],[2,4]]) + 0.2237 * e([1,[2,5]]) - 0.0269 * e([[2,6]]) - 0.0745 * e([[1,7]]) + 0.1673 * e([[1,6],2]) + 0.4487 * e([[1,5],[2,2]]) - 1.5619 * e([[1,4],[2,3]]) + 1.4290 * e([[1,3],[2,4]]) - 0.3360 * e([[1,2],[2,5]]) - 0.1117 * e([1,[2,6]]) + 0.0390 * e([[2,7]]) + 0.0271 * e([[1,8]]) + 0.3042 * e([[1,7],2]) - 1.6504 * e([[1,6],[2,2]]) + 2.2538 * e([[1,5],[2,3]]) - 0.0840 * e([[1,4],[2,4]]) - 1.9334 * e([[1,3],[2

In [12]:
# Export velocity:
oti.save(u,"u_dLx_pc100_dLy_pc100_n"+str(order)+".matso")
# Export pressure:
oti.save(pf_o2,"pf_o2_dLx_pc100_dLy_pc100_n"+str(order)+".matso")
oti.save(pf,"pf_dLx_pc100_dLy_pc100_n"+str(order)+".matso")
print("Finished!")

Finished!


In [30]:
u_loaded = oti.read("u_n10.matso")

In [31]:
u_loaded[40000,0]

-0.0048 - 0.0181 * e([1]) - 0.0021 * e([[1,2]]) + 0.0272 * e([[1,3]]) - 0.0292 * e([[1,4]]) - 0.0075 * e([[1,5]]) + 0.0590 * e([[1,6]]) - 0.0745 * e([[1,7]]) + 0.0271 * e([[1,8]]) + 0.0525 * e([[1,9]]) - 0.1036 * e([[1,10]])

In [29]:
dudx = u.get_deriv( [ [ 1, 2 ] ] )
dudy = u.get_deriv( [ [ 2, 2 ] ] )

In [ ]:
# Compute the relative errors of the derivative dudro(ri) for all computed orders.
idx = i
errors = np.zeros(order+1)

u_rerr = np.abs(u_r - u_ra)/np.abs(u_ra)

errors[0] = np.abs(u_rerr[idx,0])

for ordi in range(1,order + 1):
    dudE = u.get_deriv( [[1,ordi]] )
    dudE[0,0:2] = np.nan
    dudE_r = np.linalg.norm(dudE.real,axis=1).reshape((dudE.shape[0],-1))
    
    dudE_a = u_a.get_deriv([[1,ordi]])
    dudE_a[0,0:2] = np.nan
    dudE_ra = np.linalg.norm(dudE_a.real,axis=1).reshape((dudE_a.shape[0],-1))
    
    dudE_rerr=np.abs(dudE_r - dudE_ra)/np.abs(dudE_ra)
    
    errors[ordi] = np.abs(dudE_rerr[idx,0])
    
# end for 

In [ ]:
# np.save("errors_tri6_h01_ndofs.npy",errors)
# errors



# Errors for case of h003 tri6 132734*2 DOFs full perturbation
errors_h003 = np.array([2.56086948e-05, 1.14080377e-04, 6.85504434e-05, 8.74219077e-05,
                        1.07142953e-04, 1.27469786e-04, 1.48155015e-04, 1.69036897e-04,
                        1.90022702e-04, 2.11062481e-04, 2.32129994e-04, 2.53211624e-04,
                        2.74300506e-04, 2.95393191e-04, 3.16488122e-04, 3.37584307e-04,
                        3.58681010e-04, 3.79778507e-04, 4.00876567e-04, 4.21974759e-04,
                        4.43073730e-04, 4.64172962e-04, 4.85272692e-04, 5.06372538e-04,
                        5.27473242e-04, 5.48573676e-04, 5.69675079e-04, 5.90776585e-04,
                        6.11878232e-04, 6.32980413e-04, 6.54083090e-04])

# Errors for case of h01 tri6 40000*2 DOFs full perturbation
errors_h01_100 = np.array([3.93448745e-05, 5.97914968e-05, 7.33593831e-05, 9.19072935e-05,
       1.11951434e-04, 1.32480222e-04, 1.53166564e-04, 1.73898974e-04,
       1.94644412e-04, 2.15396108e-04, 2.36152776e-04, 2.56912591e-04,
       2.77673564e-04, 2.98434580e-04, 3.19195660e-04, 3.39957309e-04,
       3.60719762e-04, 3.81483169e-04, 4.02247282e-04, 4.23012046e-04,
       4.43777256e-04, 4.64543015e-04, 4.85309347e-04, 5.06076280e-04,
       5.26843786e-04, 5.47611897e-04, 5.68380565e-04, 5.89149764e-04,
       6.09919729e-04, 6.30690282e-04, 6.51461708e-04])

errors_h01_25 = np.array([3.93448745e-05, 1.86232659e-04, 2.63511101e-05, 1.26557130e-05,
       1.24815069e-04, 2.60426669e-04, 4.04132030e-04, 5.52018937e-04,
       7.07056611e-04, 8.82319965e-04, 1.12849128e-03, 1.53064500e-03,
       2.66506834e-03, 5.42554179e-03, 1.45118431e-02, 1.96694251e-02,
       1.39537143e-02, 9.61933711e-03, 2.47790960e+00, 8.03949359e+00,
       9.08759988e+01, 3.11822178e+02, 9.44902209e+02, 6.00574845e+03,
       3.74602300e+04, 1.29611074e+05, 4.88765443e+05, 1.97270613e+06,
       6.66010039e+06, 4.17089279e+07, 1.85374814e+08])

errors_h01_50 = np.array([3.93448745e-05, 1.02327547e-04, 9.92380262e-05, 1.42352923e-04,
       1.98890373e-04, 2.59710797e-04, 3.21950103e-04, 3.84680703e-04,
       4.47622983e-04, 5.10728329e-04, 5.74085921e-04, 6.37832001e-04,
       7.02545944e-04, 7.68770409e-04, 8.38532517e-04, 9.14159007e-04,
       9.99404754e-04, 1.10933476e-03, 1.26627513e-03, 1.48388470e-03,
       1.83543188e-03, 2.36258273e-03, 3.61074691e-03, 5.49644247e-03,
       1.00088054e-02, 1.52796563e-02, 2.83872736e-02, 4.87119055e-02,
       1.24523666e-01, 1.85081310e-01, 2.92495183e-01])

errors_h01_75 = np.array([3.93448745e-05, 7.43616483e-05, 8.89456847e-05, 1.17785517e-04,
       1.50766709e-04, 1.85065103e-04, 2.19795013e-04, 2.54666563e-04,
       2.89584089e-04, 3.24516901e-04, 3.59454604e-04, 3.94395395e-04,
       4.29341097e-04, 4.64294017e-04, 4.99255018e-04, 5.34232834e-04,
       5.69234124e-04, 6.04263902e-04, 6.39330267e-04, 6.74457215e-04,
       7.09648788e-04, 7.44959411e-04, 7.80363751e-04, 8.15978378e-04,
       8.51756609e-04, 8.87822911e-04, 9.24407671e-04, 9.61101112e-04,
       9.98597419e-04, 1.03720177e-03, 1.07652118e-03])

errors_h01_dudE = np.array([1.68228122e-05, 1.68228143e-05, 1.68228161e-05, 1.68228181e-05,
       1.68228200e-05, 1.68228213e-05, 1.68228234e-05, 1.68228253e-05,
       1.68228261e-05, 1.68228284e-05, 1.68228298e-05, 1.68228312e-05,
       1.68228327e-05, 1.68228351e-05, 1.68228366e-05, 1.68228387e-05,
       1.68228406e-05, 1.68228423e-05, 1.68228447e-05, 1.68228467e-05,
       1.68228482e-05, 1.68228496e-05, 1.68228513e-05, 1.68228531e-05,
       1.68228546e-05, 1.68228564e-05, 1.68228579e-05, 1.68228597e-05,
       1.68228622e-05, 1.68228637e-05, 5.03289566e-05])

errors

In [ ]:
from matplotlib.ticker import MultipleLocator
rc('font',**{'family':'serif','serif':['Palatino'], 'size':12})

In [ ]:
rc('font',**{'family':'serif','serif':['Palatino'], 'size':13})
plt.figure(figsize=(10,5))
new_order = 30
xvals  = range(new_order+1)#order+1)



yvals = errors_h01_100[:new_order+1]
plt.semilogy(xvals,yvals, label='100\% Perturbation')#'{0} DOFs'.format(132734*2))

yvals = errors_h01_75[:new_order+1]
plt.semilogy(xvals,yvals, label='75\% Perturbation')#'{0} DOFs'.format(132734*2))

yvals = errors_h01_50[:new_order+1]
plt.semilogy(xvals,yvals, label='50\% Perturbation')#'{0} DOFs'.format(132734*2))

yvals = errors_h01_25[:new_order+1]
plt.semilogy(xvals,yvals, label=' 25\% Perturbation')#'{0} DOFs'.format(40000*2))


# yvals = errors[:new_order+1]
# plt.semilogy(xvals,yvals, 'x-',label='{0} DOFs'.format(40000*2))

ax = plt.gca()

ax.xaxis.set_major_locator(MultipleLocator(1))

ax.set_axisbelow(True)
# plt.rc('xtick',fontsize=14)
plt.grid(which='major',color='#CCCCCC')
plt.minorticks_on()
plt.grid(which='minor',color='#EEEEEE')
ax.xaxis.set_minor_locator(MultipleLocator(1))
plt.axis([-0.5,len(xvals)-1+0.5,1e-7, 1e0])
plt.ylabel("Relative error of $\\left.\\mathbf{u}_{, r_{o}^{p}}\\right|_{r_{i}}$")
plt.xlabel("Order of derivative $p$")
plt.legend()
plt.tight_layout()

plt.savefig("imgs/error_dudro_n"+str(new_order)+".pdf",dpi=150)
plt.show()

In [ ]:
rc('font',**{'family':'serif','serif':['Palatino'], 'size':16})

plt.figure(figsize=(10,5))

grid = Th.to_pv(pd = [u_r],pd_names=['u'])
eps = 1e-2

a = [(ri.real+eps)*np.cos(np.pi/6),(ri.real+eps)*np.sin(np.pi/6),0]
b = [(ro.real-eps)*np.cos(np.pi/6),(ro.real-eps)*np.sin(np.pi/6),0]

grid.plot_over_line(a, b, resolution=100, figure=False, show=False )

ax = plt.gca()

lines = ax.get_lines()
line = lines[0]
line.set_label("OTI-FEM")
line.set_marker("x")

x = line.get_xdata()
line.set_xdata(x+ri.real)












grid = Th.to_pv(pd = [u_ra],pd_names=['ua'])

f  = grid.plot_over_line(a, b, resolution=100, figure=False, show=False )

lines = ax.get_lines()
line = lines[-1]
line.set_label("Analytic")

x = line.get_xdata()
line.set_xdata(x+ri.real)

plt.xlabel("$r$")
plt.ylabel("$\\mathbf{u}$")
# plt.title("OTI solution vs Analytic solution")
plt.title("")
plt.legend()#loc='lower right')
plt.axis([ri.real,ro.real,None,None])
plt.savefig("imgs/solution_u.pdf",dpi=150)
#plt.legend()
plt.show()

In [ ]:
x = Th.x[i,0]

In [ ]:
x30 = x**30
x30.truncate([[1,30]])

In [ ]:
t = np.linspace(0.0,1.0*ro.real,100)
orders_ti = np.array([30,20,15,10,8,6,4,3,2,1],dtype=np.uint8)
error_ti = np.zeros((orders_ti.size,t.size))

kk=0

ui = u[i]
xi = Th.x[i]
yi = Th.y[i]

for ordi in orders_ti:
    u_for_ti = ui.truncate([[1,ordi+1]])
    k = 0
    for delta in t:
        u_r_ti = np.linalg.norm(u_for_ti.taylor_integrate([1],[delta]).real,axis=1).reshape((ui.shape[0],-1))
#         ux_a_ti, uy_a_ti = analytic_solution_xy(Th.x.taylor_integrate([1],[delta]).real,
#                                             Th.y.taylor_integrate([1],[delta]).real,
#                                             E.real,nu.real,ri.real,Pi.real,
#                                             ro.taylor_integrate([1],[delta]).real,Po.real)
        ux_a_ti, uy_a_ti = analytic_solution_xy(xi.taylor_integrate([1],[delta]).real,
                                            yi.taylor_integrate([1],[delta]).real,
                                            E.real,nu.real,
                                            ri.real,
                                            Pi.real,
                                            ro.taylor_integrate([1],[delta]).real,
                                            Po.real)
        u_a_ti = np.zeros(ui.shape)
        u_a_ti[:,0:1]=ux_a_ti
        u_a_ti[:,1:2]=uy_a_ti
        u_ra_ti = np.linalg.norm(u_a_ti.real,axis=1).reshape((u_a_ti.shape[0],-1))

        error_ti[kk,k]=abs( (u_r_ti[0,0]-u_ra_ti[0,0])/(u_ra_ti[0,0]) )

        k+=1

    #
    kk+=1
error_ti

In [ ]:
rc('font',**{'family':'serif','serif':['Palatino'], 'size':16})

plt.figure(figsize=(10,5))
norders = len(orders_ti)
for kk in range(norders):
    plt.semilogy(100*t/ro.real,error_ti[-1-kk,:],label="Order "+str(orders_ti[-1-kk]))

plt.ylabel("Relative error of $\\left.\\mathbf{u}(r_o+\\Delta r_o)\\right|_{r_i}$")
plt.xlabel("$\\displaystyle{\\frac{\Delta r_o}{r_o}} [ \% ]$")
plt.legend()

ax = plt.gca()

ax.xaxis.set_major_locator(MultipleLocator(10))

ax.set_axisbelow(True)
# plt.rc('xtick',fontsize=14)
plt.grid(which='major',color='#CCCCCC')
plt.minorticks_on()
plt.grid(which='minor',color='#EEEEEE')

ax.xaxis.set_minor_locator(MultipleLocator(2))
# plt.axis([-0.5,len(xvals)-1+0.5,1e-7, 1e-1])
# plt.ylabel("Relative error of $\mathbf{u}_{, r_{o}^{p}}(r_{i})$")
# plt.xlabel("Order of derivative $p$")
plt.axis([None,100*1.3,1e-6,1e0])
plt.tight_layout()
plt.savefig("imgs/error_ti.pdf",dpi=150)
plt.show()

In [ ]:
tr = np.linspace( -1.0*ro.real,1.0*ro.real, 200 )
tE = np.linspace( -1.0*E.real, 1.0*E.real,  200 )

Tr, TE = np.meshgrid(tr,tE)


orders_ti = np.array([30,20,15,10,8,6,4,3,2,1],dtype=np.uint8)
# orders_ti = np.array([5,],dtype=np.uint8)

error_ti = np.zeros((orders_ti.size,TE.shape[0],TE.shape[1]))

kk=0

ui = u[i]
xi = Th.x[i]
yi = Th.y[i]

for ordi in orders_ti:
    
    u_for_ti = ui.copy()
    # Truncate all ordi + 1 values 
    u_for_ti = u_for_ti.truncate([[1,ordi+1]])
    u_for_ti = u_for_ti.truncate([[2,ordi+1]])
    for oo in range(1,ordi+1):
        o1 = oo
        o2 = ordi+1-oo
#         print(o1, o2, o1+o2)
        u_for_ti = u_for_ti.truncate([[1,o1],[2,o2]])
    
#     print(u_for_ti)
#     print(u_for_ti)
    for ii in range(TE.shape[0]):
        for jj in range(TE.shape[1]):
            dr = Tr[ii,jj]
            dE = TE[ii,jj]
#             print(ii,jj)
            u_r_ti = np.linalg.norm(u_for_ti.taylor_integrate([1,2],[dr,dE]).real,axis=1).reshape((ui.shape[0],-1))
#         ux_a_ti, uy_a_ti = analytic_solution_xy(Th.x.taylor_integrate([1],[delta]).real,
#                                             Th.y.taylor_integrate([1],[delta]).real,
#                                             E.real,nu.real,ri.real,Pi.real,
#                                             ro.taylor_integrate([1],[delta]).real,Po.real)
            ux_a_ti, uy_a_ti = analytic_solution_xy(xi.taylor_integrate([1,2],[dr,dE]).real,
                                                yi.taylor_integrate([1,2],[dr,dE]).real,
                                                E.taylor_integrate([1,2],[dr,dE]).real,
                                                nu.real,
                                                ri.real,
                                                Pi.real,
                                                ro.taylor_integrate([1,2],[dr,dE]).real,
                                                Po.real)
            u_a_ti = np.zeros(ui.shape)
            u_a_ti[:,0:1]=ux_a_ti
            u_a_ti[:,1:2]=uy_a_ti
            u_ra_ti = np.linalg.norm(u_a_ti.real,axis=1).reshape((u_a_ti.shape[0],-1))

            error_ti[kk,ii,jj]=abs( (u_r_ti[0,0]-u_ra_ti[0,0])/(u_ra_ti[0,0]) )

        

    #
    kk+=1
    
error_ti

In [ ]:
error_ti.shape

In [ ]:
error_ti.max()

In [ ]:
from matplotlib import ticker, cm, colors

fig = plt.figure(figsize=(8,6))
lev_exp = np.array([-7,-6,-5,-4,-3,-2,-1,0,1,2,14],dtype=np.float64)
levs = np.power(10, lev_exp)
# cs = ax.contourf(X, Y, z, levs, norm=colors.LogNorm())

# cp = plt.contourf(100*Tr/ro.real,100*TE/E.real,error_ti[0], levels=16,locator=ticker.LogLocator(10),norm=colors.LogNorm(vmax = 1e1), cmap=cm.jet)
cp = plt.contourf(100*Tr/ro.real,100*TE/E.real,error_ti[0], levs,antialiased=True, norm=colors.LogNorm(vmax = 1e1), cmap=cm.jet)
cbar = fig.colorbar(cp)

CS=plt.contour(100*Tr/ro.real,100*TE/E.real,error_ti[0], [1e-4])

plt.clabel(CS,fmt="%.1e", inline=1, fontsize=10)

cbar.ax.set_ylabel("Relative error of \n$\\left.\\mathbf{u}(r_o+\\Delta r_o, E+\\Delta E)\\right|_{r_i}$")
plt.xlabel("$\\displaystyle{\\frac{\Delta r_o}{r_o}} [ \% ]$")
plt.ylabel("$\\displaystyle{\\frac{\Delta E}{E}} [ \% ]$")


ax = plt.gca()
ax.xaxis.set_major_locator(MultipleLocator(20))
ax.yaxis.set_major_locator(MultipleLocator(20))
plt.grid(color='#222222', linewidth=0.5)
# plt.axis([-50,50,-100,100])

plt.tight_layout()
plt.savefig('imgs/error_ti_Ero.pdf',dpi=150)
plt.show()

In [ ]:
from matplotlib import ticker, cm, colors

rc('font',**{'family':'serif','serif':['Palatino'], 'size':16})
             
fig = plt.figure(figsize=(8,6))
lev_exp = np.array([-7,-6,-5,-4,-3,-2,-1,0,1,2,14],dtype=np.float64)
levs = np.power(10, lev_exp)
# cs = ax.contourf(X, Y, z, levs, norm=colors.LogNorm())

# cp = plt.contourf(100*Tr/ro.real,100*TE/E.real,error_ti[0], levels=16,locator=ticker.LogLocator(10),norm=colors.LogNorm(vmax = 1e1), cmap=cm.jet)
# cp = plt.contourf(100*Tr/ro.real,100*TE/E.real,error_ti[0], levs,antialiased=True, norm=colors.LogNorm(vmax = 1e1), cmap=cm.jet)
# cbar = fig.colorbar(cp)

for kk in range(len(orders_ti)):
    CS=plt.contour(100*Tr/ro.real,100*TE/E.real,error_ti[-1-kk], [1e-4], colors = 'C'+str(kk))
    
#     plt.clabel(CS,fmt='Order '+str(orders_ti[kk]), inline=1, fontsize=12, colors='k')
#     plt.clabel(CS,fmt='Ord. '+str(orders_ti[-1-kk]), inline=1, fontsize=8, colors='k')
    
    CS.collections[0].set_label('Order '+str(orders_ti[-1-kk]))

# cbar.ax.set_ylabel("Relative error of \n$\\left.\\mathbf{u}(r_o+\\Delta r_o, E+\\Delta E)\\right|_{r_i}$")
plt.xlabel("$\\displaystyle{\\frac{\Delta r_o}{r_o}} [ \% ]$")
plt.ylabel("$\\displaystyle{\\frac{\Delta E}{E}} [ \% ]$")


ax = plt.gca()
ax.xaxis.set_major_locator(MultipleLocator(20))
ax.xaxis.set_minor_locator(MultipleLocator(10))
ax.yaxis.set_major_locator(MultipleLocator(20))
ax.yaxis.set_minor_locator(MultipleLocator(10))
plt.grid(which='major',color='#CCCCCC', linewidth=0.5)
plt.minorticks_on()
plt.grid(which='minor',color='#EEEEEE')

# plt.axis([-50,50,-100,100])
plt.legend()
plt.tight_layout()
plt.savefig('imgs/error_ti_Ero_orders.pdf',dpi=150)
plt.show()

In [ ]:
rc('font',**{'family':'serif','serif':['Palatino'], 'size':16})

plt.figure(figsize=(10,5))






delta = 1.0







u_r_ti = np.linalg.norm(u.taylor_integrate([1],[delta]).real,axis=1).reshape((u.shape[0],-1))


grid = Th.to_pv(pd = [u_r_ti],pd_names=['u'])
eps = 1e-2

a = [(ri.real+eps)*np.cos(np.pi/6),(ri.real+eps)*np.sin(np.pi/6),0]
b = [(ro.real-eps)*np.cos(np.pi/6),(ro.real-eps)*np.sin(np.pi/6),0]

grid.plot_over_line(a, b, resolution=100, figure=False, show=False )

ax = plt.gca()

lines = ax.get_lines()
line = lines[0]
line.set_label("OTI-FEM-TI")
line.set_marker("x")

x = line.get_xdata()
line.set_xdata(x*(1+delta)+ri.real)










ux_a_ti, uy_a_ti = analytic_solution_xy(Th.x.taylor_integrate([1],[delta]).real,
                                        Th.y.taylor_integrate([1],[delta]).real,
                                        E.real,nu.real,ri.real,Pi.real,
                                        ro.taylor_integrate([1],[delta]).real,Po.real)
u_a_ti = np.zeros(u.shape)
u_a_ti[:,0:1]=ux_a_ti
u_a_ti[:,1:2]=uy_a_ti
u_ra_ti = np.linalg.norm(u_a_ti.real,axis=1).reshape((u_a.shape[0],-1))

















grid = Th.to_pv(pd = [u_ra_ti],pd_names=['ua'])

f  = grid.plot_over_line(a, b, resolution=100, figure=False, show=False )

lines = ax.get_lines()
line = lines[-1]
line.set_label("Analytic")

x = line.get_xdata()
line.set_xdata(x*(1+delta)+ri.real)













grid = Th.to_pv(pd = [u_ra],pd_names=['ua'])

f  = grid.plot_over_line(a, b, resolution=100, figure=False, show=False )

lines = ax.get_lines()
line = lines[-1]
line.set_label("Original solution $\mathbf{u}(r_o)$")

x = line.get_xdata()
line.set_xdata(x+ri.real)
line.set_linestyle('--')















plt.xlabel("$r$")
plt.ylabel("$\\mathbf{{u}}(r_o+{0})$".format(delta))
# plt.title("OTI solution vs Analytic solution")
plt.title("")
plt.legend(loc='upper center', ncol=3)
plt.axis([ri.real,ro.real+delta,None,None])
plt.savefig("imgs/solution_u_ti_delta_{0}.pdf".format(delta),dpi=150)
#plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,5))


new_order = 10

dudE = u.get_deriv( [[1,new_order]] )
dudE[0,0:2] = np.nan
dudE_r = np.linalg.norm(dudE.real,axis=1).reshape((dudE.shape[0],-1))
dudE_r



dudE_a = u_a.get_deriv([[1,new_order]])
dudE_a[0,0:2] = np.nan
dudE_ra = np.linalg.norm(dudE_a.real,axis=1).reshape((dudE_a.shape[0],-1))






grid = Th.to_pv(pd = [dudE_r],pd_names=['oti'])

eps = 1e-2

a = [(ri.real+eps)*np.cos(np.pi/6),(ri.real+eps)*np.sin(np.pi/6),0]
b = [(ro.real-eps)*np.cos(np.pi/6),(ro.real-eps)*np.sin(np.pi/6),0]

grid.plot_over_line(a, b, resolution=100, figure=False, show=False )
ax = plt.gca()

lines = ax.get_lines()
line = lines[0]
line.set_label("OTI-FEM")
line.set_marker("x")


x = line.get_xdata()
line.set_xdata(x+ri.real)





grid = Th.to_pv(pd = [dudE_ra],pd_names=['analytic'])

f  = grid.plot_over_line(a, b, resolution=100, figure=False, show=False )
ax = plt.gca()
lines = ax.get_lines()
line = lines[-1]
line.set_label("Analytic")
x = line.get_xdata()
line.set_xdata(x+ri.real)


# Rescale x axis
# for line in lines:
    



plt.xlabel("$r$")
# plt.ylabel("30th order derivative of u w.r.t. E")
# plt.title("OTI solution vs Analytic solution")
plt.title("")
str_order = " "
if new_order != 1:
    str_order = str(new_order)
# end if 
plt.ylabel("$\\mathbf{u}_{,r_{o}^{"+str_order+"}}$")
plt.legend()#loc='upper right')
plt.axis([ri.real,ro.real,None,None])
plt.savefig('imgs/solution_dudro_n'+str(new_order)+'.pdf',dpi=150)
plt.show()

In [35]:
import pyvista as pv
p = pv.BackgroundPlotter()
args_cbar = dict(height=0.75, vertical=True, position_x=0.05, 
                 position_y=0.05, interactive=False,
                 title_font_size=30, label_font_size=30)

In [22]:
# p = pv.Plotter()
p.set_background("white")

grid = Th.to_pv(dims = 2, pd = [u.real],pd_names=['u'])

factor = 100
p.clear()

p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet', scalar_bar_args=args_cbar)
# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,color='grey', scalar_bar_args=args_cbar)
# p.add_mesh(arrows, lighting=False, stitle="u", scalar_bar_args=args_cbar)
# p.update_coordinates(grid.points + u*factor)

strmline1 = grid.streamlines(
    integration_direction = 'both',
    pointa = (0.5,0,0),
    pointb = (0.5,1,0),   
    n_points = 20
)
strmline2 = grid.streamlines(
    integration_direction = 'both',
    pointa = (0,0,0),
    pointb = (0.1,0.1,0),   
    n_points = 10
)
strmline3 = grid.streamlines(
    integration_direction = 'both',
    pointa = (0.9,0.1,0),
    pointb = (1,0,0),   
    n_points = 10
)

p.add_mesh(strmline1, show_edges=False, line_width=2,color='gray')
p.add_mesh(strmline2, show_edges=False, line_width=2,color='gray')
p.add_mesh(strmline3, show_edges=False, line_width=2,color='gray')

p.show_bounds()
p.show_grid()
# p.screenshot(filename='test2.jpeg',window_size= [1024*2, 768*2])

In [33]:
Th.export("test.vtk",dims = 2, pd = [pf_o2.real,u.real],pd_names=['P','u'])

In [56]:
# p = pv.Plotter()
p.set_background("white")

grid = Th.to_pv(dims = 2, pd = [pf_o2.real],pd_names=['P'])

factor = 100
p.clear()

p.add_mesh(grid, show_edges=False, line_width=2, grid = True,categories=64,cmap='gist_ncar', scalar_bar_args=args_cbar)
# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,color='grey', scalar_bar_args=args_cbar)
# p.add_mesh(arrows, lighting=False, stitle="u", scalar_bar_args=args_cbar)
# p.update_coordinates(grid.points + u*factor)

# strmline1 = grid.streamlines(
#     integration_direction = 'both',
#     pointa = (0.5,0,0),
#     pointb = (0.5,1,0),   
#     n_points = 20
# )
# strmline2 = grid.streamlines(
#     integration_direction = 'both',
#     pointa = (0,0,0),
#     pointb = (0.1,0.1,0),   
#     n_points = 10
# )
# strmline3 = grid.streamlines(
#     integration_direction = 'both',
#     pointa = (0.9,0.1,0),
#     pointb = (1,0,0),   
#     n_points = 10
# )

# p.add_mesh(strmline1, show_edges=False, line_width=2,color='gray')
# p.add_mesh(strmline2, show_edges=False, line_width=2,color='gray')
# p.add_mesh(strmline3, show_edges=False, line_width=2,color='gray')

p.show_bounds()
p.show_grid()
# p.screenshot(filename='test2.jpeg',window_size= [1024*2, 768*2])

In [14]:
p2 = pv.Plotter()

In [58]:
# p = pv.Plotter()
# p2 = pv.Plotter()
p2 = pv.BackgroundPlotter()

p2.set_background("white")
dx = 1.0
dy = 1.0
Th2.x = Th.x.taylor_integrate([1,2],[dx,dy])
Th2.y = Th.y.taylor_integrate([1,2],[dx,dy])
u2 = u.taylor_integrate([1,2],[dx,dy])
grid = Th2.to_pv(dims = 2, pd = [u2.real],pd_names=['u'])

factor = 100
p2.clear()

p2.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet', scalar_bar_args=args_cbar)
# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,color='grey', scalar_bar_args=args_cbar)
# p.add_mesh(arrows, lighting=False, stitle="u", scalar_bar_args=args_cbar)
# p.update_coordinates(grid.points + u*factor)

xmax = np.max(Th2.x.real) 
xmin = np.min(Th2.x.real) 
hhx  = xmax-xmin
xmid = 0.5*(xmax+xmin)

ymax = np.max(Th2.y.real) 
ymin = np.min(Th2.y.real) 
hhy  = ymax-ymin
ymid = 0.5*(ymax+ymin)

strmline1 = grid.streamlines(
    integration_direction = 'both',
    pointa = (xmid,ymin,0),
    pointb = (xmid,ymax,0),   
    n_points = 20
)
strmline2 = grid.streamlines(
    integration_direction = 'both',
    pointa = (xmin,0,0),
    pointb = (xmin+hhx*.1,ymin+hhy*.1,0),   
    n_points = 10
)
strmline3 = grid.streamlines(
    integration_direction = 'both',
    pointa = (xmax-hhx*0.1,ymin+hhy*0.1,0),
    pointb = (xmax,ymin,0),   
    n_points = 10
)

p2.add_mesh(strmline1, show_edges=False, line_width=2,color='gray')
p2.add_mesh(strmline2, show_edges=False, line_width=2,color='gray')
p2.add_mesh(strmline3, show_edges=False, line_width=2,color='gray')

p2.show_bounds()
p2.show_grid()
fname = 'u_ti_ord_'+str(order)+'_dx_'+str(dx)+'_dy_'+str(dy)+'.jpeg'
p2.screenshot(filename=fname,window_size= [1024*2, 768*2])

In [59]:
[ux2, uy2, pf2 ],K,f = solve_2d_cavity(Th2, utop, solver=solver)
u22 = alg.zeros((ux2.shape[0],3))
u22[:,0:1] = ux2
u22[:,1:2] = uy2

Setting up problem

Starting elemental computations.

Finished assembly.
Setting up boundary condition.

Ended boundary condition setup.
Starting system solution.

Converted matrix to csr format. - Starting solution process.

Solved problem.
Total run time: 7.220603 s 



In [60]:
p2 = pv.BackgroundPlotter()

grid = Th2.to_pv(dims = 2, pd = [u22.real],pd_names=['u2'])

factor = 100
p2.clear()

p2.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet', scalar_bar_args=args_cbar)
# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,color='grey', scalar_bar_args=args_cbar)
# p.add_mesh(arrows, lighting=False, stitle="u", scalar_bar_args=args_cbar)
# p.update_coordinates(grid.points + u*factor)

xmax = np.max(Th2.x.real) 
xmin = np.min(Th2.x.real) 
hhx  = xmax-xmin
xmid = 0.5*(xmax+xmin)

ymax = np.max(Th2.y.real) 
ymin = np.min(Th2.y.real) 
hhy  = ymax-ymin
ymid = 0.5*(ymax+ymin)

strmline1 = grid.streamlines(
    integration_direction = 'both',
    pointa = (xmid,ymin,0),
    pointb = (xmid,ymax,0),   
    n_points = 20
)
strmline2 = grid.streamlines(
    integration_direction = 'both',
    pointa = (xmin,0,0),
    pointb = (xmin+hhx*.1,ymin+hhy*.1,0),   
    n_points = 10
)
strmline3 = grid.streamlines(
    integration_direction = 'both',
    pointa = (xmax-hhx*0.1,ymin+hhy*0.1,0),
    pointb = (xmax,ymin,0),   
    n_points = 10
)

p2.add_mesh(strmline1, show_edges=False, line_width=2,color='gray')
p2.add_mesh(strmline2, show_edges=False, line_width=2,color='gray')
p2.add_mesh(strmline3, show_edges=False, line_width=2,color='gray')

p2.show_bounds()
p2.show_grid()
fname = 'u_ti_ord_'+str(order)+'_dx_'+str(dx)+'_dy_'+str(dy)+'.jpeg'
# p2.screenshot(filename=fname,window_size= [1024*2, 768*2])

In [61]:
p2 = pv.BackgroundPlotter()
u2_norm = np.linalg.norm( u2.real,axis=1)
u22_norm= np.linalg.norm(u22.real,axis=1)

uerr = np.abs((u2_norm-u22_norm)/u22_norm)
grid = Th2.to_pv(dims = 2, pd = [uerr],pd_names=['Relative Error'])

factor = 100
p2.clear()

p2.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet', scalar_bar_args=args_cbar, log_scale = True)



p2.show_bounds()
p2.show_grid()
# fname = 'u_ti_ord_'+str(order)+'_dx_'+str(dx)+'_dy_'+str(dy)+'.jpeg'
# p2.screenshot(filename=fname,window_size= [1024*2, 768*2])

In [52]:
p2 = pv.BackgroundPlotter()

grid = Th2.to_pv(dims = 2, pd = [np.abs( u2.real - u22.real)],pd_names=['Absolute Error'])

factor = 100
p2.clear()

p2.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet', scalar_bar_args=args_cbar)



p2.show_bounds()
p2.show_grid()
# fname = 'u_ti_ord_'+str(order)+'_dx_'+str(dx)+'_dy_'+str(dy)+'.jpeg'
# p2.screenshot(filename=fname,window_size= [1024*2, 768*2])

In [14]:
p.set_background("white")

u = alg.sqrt(ux**2 + uy**2)

grid = Th.to_pv( dims=2, pd = [u.real],pd_names=['u'])

factor = 100
p.clear()

p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args=args_cbar)
# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,color='grey', scalar_bar_args=args_cbar)
# p.add_mesh(arrows, lighting=False, stitle="u", scalar_bar_args=args_cbar)
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()


In [19]:
p.set_background("white")

grid = Th.to_pv(pd = [pf_o2.real],pd_names=['p'])

factor = 100
p.clear()

p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args=args_cbar)
# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,color='grey', scalar_bar_args=args_cbar)
# p.add_mesh(arrows, lighting=False, stitle="u", scalar_bar_args=args_cbar)
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()


In [26]:
ux.real

array([[5.00000000e-31],
       [4.07301817e-50],
       [5.00000000e-31],
       ...,
       [8.20382787e-05],
       [5.47039382e-05],
       [2.73554955e-05]])

In [ ]:
grid.streamlines()

In [ ]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
# pv.set_plot_theme('default')
p.set_background("white")
grid = Th.to_pv(pd = [u_a.real],pd_names=['u'])
factor = 100
p.clear()

# arrows = grid.glyph(scale="u", orient="u", factor = 100,tolerance=0.001)

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args=args_cbar)
# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,color='grey', scalar_bar_args=args_cbar)
# p.add_mesh(arrows, lighting=False, stitle="u", scalar_bar_args=args_cbar)
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()


In [ ]:
arrows

In [ ]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
# pv.set_plot_theme('default')
p.set_background("white")
grid = Th.to_pv(pd = [u.real],pd_names=['u'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args=args_cbar)
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [ ]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [u_a.real],pd_names=['u_analytic'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [ ]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [np.abs((u_r-u_ra)/u_ra)],pd_names=['relative error'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet')#, scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [ ]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [u.real-u_a.real],pd_names=['absolute error'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet')#, scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [ ]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [u.real-u_a.real],pd_names=['absolute error'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet')#, scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [ ]:
import pyvista as pv
p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [u_rerr],pd_names=['relative error'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet')#, scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [ ]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [dudEc],pd_names=['dudE_oti'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [ ]:
# import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [dudE_analytic],pd_names=['dudE_analytic'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()

In [ ]:
import pyvista as pv
p = pv.BackgroundPlotter()
pv.set_plot_theme('document')
grid = Th.to_pv(pd = [dudE-dudE_analytic],pd_names=['10th deriv  abs. error.'])
factor = 100
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args={'interactive':True})
# p.update_coordinates(grid.points + u*factor)

p.show_bounds()
p.show_grid()